# Extract and Transform the data from the CSV
three dataframes: df_landscape, df_caption, df_start_end_pos will be created from the csv-file.

In [14]:
import pandas as pd
import os
import numpy as np
import csv

In [13]:

# Read whole csv file and create a dataframe
csv_filename = "S_001_Daten.csv"
path_to_csv = os.path.join("SEARCH_001", "S001", csv_filename)
df_total = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", header=None)

# seperate the dataframe in three subdataframes
## df_landscape / np_landscape
endrow_landscape_data = df_total.loc[pd.isna(df_total[0])].index[0]         # get index of a row containing NaN in column 0
df_landscape = df_total[:endrow_landscape_data]                             # get landscape part of data
df_landscape = df_landscape.astype(float)                                   # clean data
np_landscape = df_landscape.to_numpy().T                                    # to numpy

## df_caption
startrow_caption_data = endrow_landscape_data + 1
endrow_caption_data = df_total.loc[pd.isna(df_total[0])].index[1]            # get index of a row containing NaN in column 0
nrows_caption_data = endrow_caption_data - startrow_caption_data - 1
df_caption = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", skiprows=startrow_caption_data, nrows=nrows_caption_data, usecols=["Code", "Bezeichnung", "Kosten"])
df_caption["Code"] = df_caption["Code"].astype(int)                         # clean data
df_caption["Bezeichnung"] = df_caption["Bezeichnung"].astype(str)           # clean data
df_caption["Kosten"] = df_caption["Kosten"].astype(float)                   # clean data

## df_start_end_pos
startrow_pos_data = endrow_caption_data + 1                                 # get index of a row where start and endpoint information starts
df_start_end_pos = pd.read_csv(path_to_csv, sep=";", encoding="unicode-escape", skiprows=startrow_pos_data, usecols=["Startpunkt", "Endpunkt"])
df_start_end_pos.index = ["x", "y"]                                         # transform dataframe
df_start_end_pos["Startpunkt"] = df_start_end_pos["Startpunkt"].astype(int)             # clean data
df_start_end_pos["Endpunkt"] = df_start_end_pos["Endpunkt"].astype(int)              # clean data
np_start_end_pos = df_start_end_pos.to_numpy().T                            # to numpy

NameError: name 'pd' is not defined

In [12]:
df_landscape.shape # (number_row, number_col)
df_landscape

NameError: name 'df_landscape' is not defined

In [8]:
df_caption

,Code,Bezeichnung,Kosten
0,0,Ebene,3.0
1,1,Fluss,5.0
2,2,Weg,1.0
3,3,Wald,4.0
4,4,Bruecke,2.0
5,5,Felswand,20.0


In [9]:
df_start_end_pos

,Startpunkt,Endpunkt
x,12,3
y,4,6


# Instantiate Grid

In [1]:
import grid

In [2]:
landscape = grid.Grid()

In [3]:
landscape.np_grid[0][14]

2.0

In [17]:
# test start end
print("start: ", landscape.start_pos)
print("end: ", landscape.end_pos)

# test name, cost and code of pos
pos = (0, 14)
print("code: ", landscape.get_code_of_pos(pos))
print("name: ", landscape.get_name_of_pos(pos))
print("cost: ", landscape.get_cost_of_pos(pos))


start:  (12, 4)
end:  (3, 6)
code:  2.0
name:  Weg
cost:  1.0


In [18]:
pos = (0,0)
neighbours = landscape.get_neighbours_of_pos(pos)
for neighbour in neighbours:
    print(neighbour)
    print("code: ", landscape.get_code_of_pos(neighbour))
    print("name: ", landscape.get_name_of_pos(neighbour))
    print("cost: ", landscape.get_cost_of_pos(neighbour))
    print("----")

(1, 0)
code:  1.0
name:  Fluss
cost:  5.0
----
(0, 1)
code:  5.0
name:  Felswand
cost:  20.0
----


# Instantiate nodes and add it to the graph

In [7]:
landscape.pos_grid[0][1]

(0, 1)

# A* Algorithmus

In [92]:
knots_to_knot_s2 = {"s0": 2, "s1": 3, "s2":4}
knots_to_goal_s4 = {"s2": 5, "s3": 9, "s4":10}

"""get_cost
Get costs of a path of knots.

Params:
    * knots (dictionary): name of knot as key and their cost as value
Returns:
    * int: sum of costs of all the given knots
"""
def get_cost(knots):
    sum = 0
    for el in knots:
        sum += knots[el]
    return sum

get_cost(knots_to_knot_s2)
get_cost(knots_to_goal_s4)

24

In [93]:
open = {}

def calculate_f(knot):
    get_cost(knots_to_knot_s2)

"""add_to_open
Adds knot with cost f(s) to the open dictionary
"""
def add_to_open(knot, cost):
    open[knot] = cost

"""get_min_from_open
Get knot from the dictionary open with minimal f(s)
"""
def get_min_from_open():
    return min(open, key=open.get)

add_to_open("s1", 3)
add_to_open("s2", 6)
add_to_open("s3", 2)
add_to_open("s3", 12)
add_to_open("s1", 1)

get_min_from_open()

's1'

In [137]:
import node
cost = 3
n1 = node.Node(cost=3, pos=(2,3))
n2 = node.Node(cost=4, pos=(3,3))
n3 = node.Node(cost=5, pos=(5,3))
adjac_list = {}
adjac_list[n1] = [n2, n3]

print(adjac_list)
for n in adjac_list.keys():
    print(n.pos)
    for v in adjac_list[n]:
        print(v.pos)

{<node.Node object at 0x00000248295707F0>: [<node.Node object at 0x0000024829570A30>, <node.Node object at 0x0000024829570D90>]}
(2, 3)
(3, 3)
(5, 3)


In [160]:
### to tuple
from numpy import array
a = np.array([[1, 2],
           [3, 4]])
print(a)
tuple(tuple(a_m.tolist()) for a_m in a )

[[1 2]
 [3 4]]


((1, 2), (3, 4))

[1, 2, 3]